In [8]:
import pandas as pd
import numpy as np
import datetime as datetime
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', 500)

In [9]:
# Load data
Order_df = pd.read_csv('order_brush_order.csv',dtype={'bank_account':np.datetime64}) 
df = Order_df.copy()
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [10]:
by_shopid = df.groupby(['shopid', 'event_time', 'userid']).count().reset_index()
by_shopid.head()
by_shopid.info()

,shopid,event_time,userid,orderid
0,10009,2019-12-27 03:06:50,196962305,1
1,10051,2019-12-27 19:16:11,2854032,1
2,10051,2019-12-29 01:56:19,48600461,1
3,10061,2019-12-28 09:27:55,168750452,1
4,10061,2019-12-28 12:05:32,194819216,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   shopid      222750 non-null  int64 
 1   event_time  222750 non-null  object
 2   userid      222750 non-null  int64 
 3   orderid     222750 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


In [11]:
by_shopid['event_time'] = pd.to_datetime(by_shopid['event_time'])
by_shopid['time_end'] = by_shopid['event_time'] + datetime.timedelta(hours=1)
by_shopid['userid'] = by_shopid['userid'].apply(str)
by_shopid.head()
by_shopid.info()

,shopid,event_time,userid,orderid,time_end
0,10009,2019-12-27 03:06:50,196962305,1,2019-12-27 04:06:50
1,10051,2019-12-27 19:16:11,2854032,1,2019-12-27 20:16:11
2,10051,2019-12-29 01:56:19,48600461,1,2019-12-29 02:56:19
3,10061,2019-12-28 09:27:55,168750452,1,2019-12-28 10:27:55
4,10061,2019-12-28 12:05:32,194819216,1,2019-12-28 13:05:32


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   shopid      222750 non-null  int64         
 1   event_time  222750 non-null  datetime64[ns]
 2   userid      222750 non-null  object        
 3   orderid     222750 non-null  int64         
 4   time_end    222750 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 8.5+ MB


In [12]:
my_submission = pd.DataFrame([], columns=['shopid', 'userid'])
num = 0
for shop_id in by_shopid['shopid'].unique():
    # if num >= 20:
    #     break
    # num += 1
    # if shop_id != 10287:
    #     continue

    tmp_df = by_shopid[by_shopid['shopid'] == shop_id]
    merge_df = pd.merge(tmp_df.assign(key='shopid'), tmp_df.assign(key='shopid'), on='key').drop('key', axis=1)
    final_df = merge_df[(merge_df['event_time_x'] <= merge_df['event_time_y']) & (merge_df['time_end_x'] >= merge_df['event_time_y'])]
    tmp_df = None
    merge_df = None

    final_df = final_df.drop(['userid_x', 'orderid_x', 'time_end_x', 'shopid_y', 'event_time_y', 'time_end_y'], axis=1)
    res = final_df.groupby(['shopid_x', 'event_time_x', 'userid_y']).sum()
    res = res[res.orderid_y >= 3]
    # print(res)
    if res.shape[0] > 0:
        res = res - res.mean(level='event_time_x')
        # print(res_max)
        res = res[res.orderid_y >= 0]
    res.reset_index(inplace=True)
    final_df = None
    if res.shape[0] == 0:
        res = pd.DataFrame({"shopid": [shop_id], "userid": ['0']})
    else:
        res['userid_y'] = res[['shopid_x', 'userid_y']].groupby(['shopid_x'])['userid_y'].transform(lambda x: '&'.join(x))
        res = res[['shopid_x', 'userid_y']].drop_duplicates()
        res.columns = ['shopid', 'userid']
    # print(res)
    my_submission = my_submission.append(res, ignore_index=True)
    # my_submission.update(zip(res))
    # df['text'] = df[['name','text','month']].groupby(['name','month'])['text'].transform(lambda x: ','.join(x))

In [13]:
# drop duplicate userid
for idx, shop_id in enumerate(my_submission['shopid']):
    if my_submission['userid'][idx] == '0':
        continue
    else:
        val = my_submission['userid'][idx]
        split_lst = list(set(val.split('&')))
        # print(split_lst)
        my_submission['userid'][idx] = split_lst
my_submission['userid'] = my_submission['userid'].str.join("&") 

In [14]:
my_submission.to_csv('submission_ray.csv', index=False, header=True)